This notebook will demonstrate how to use the **constrained training algorithms** implemented in this toolkit.

To train a network, instantiate an algorithm, passing to it the model, the dataset, a list of `FairnessConstraint`s and the algorithm's hyperparameters.

Load and prepare data from `folktables`:

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from folktables import ACSDataSource, ACSIncome

device = 'cpu'
torch.set_default_device(device)

# load folktables data
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["OK"], download=True)
features, labels, groups = ACSIncome.df_to_numpy(acs_data)
# split
X_train, X_test, y_train, y_test, groups_train, groups_test = train_test_split(
    features, labels, groups, test_size=0.2, random_state=42)
# scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# make into a pytorch dataset, remove the sensitive attribute (RAC1P)
features_train = torch.tensor(X_train, dtype=torch.float32)[:,:-1].to(device)
labels_train = torch.tensor(y_train,dtype=torch.float32).to(device)
dataset_train = torch.utils.data.TensorDataset(features_train, labels_train)

c_batch_size = 128
min_subgroup_size = c_batch_size
# For each subgroup, FairnessConstraint needs a list of indices of samples belonging to that subgroup
group_indices_train = [
    np.nonzero(groups_train == group_id)[0] for group_id in np.unique(groups_train)
    if np.count_nonzero(groups == group_id) > min_subgroup_size
]

# repeat for test set
features_test = torch.tensor(X_test, dtype=torch.float32)[:,:-1].to(device)
labels_test = torch.tensor(y_test,dtype=torch.float32).to(device)
dataset_test = torch.utils.data.TensorDataset(features_test, labels_test)
group_indices_test = [
    np.nonzero(groups_test == group_id)[0] for group_id in np.unique(groups_test)
    if np.count_nonzero(groups == group_id) > min_subgroup_size
]

print(f'Protected attribute: {ACSIncome.group}')
print(f'Number of subgroups considered: {len(group_indices_train)}')
print(f'Size of subgroups: {[len(g) for g in group_indices_train]}')

Protected attribute: RAC1P
Number of subgroups considered: 6
Size of subgroups: [10685, 794, 1213, 259, 315, 997]


Let's say we want to add an equal loss constraint on the model.

We can do that by using the `FairnessConstraint` class, which will handle sampling (if possible, sampling an equal number of samples from each relevant subgroup in each minibatch), and passing it the function that will calculate the value of the constraint.

In [ ]:
from itertools import combinations
from humancompatible.train.constraints.constraint import FairnessConstraint
from humancompatible.train.constraints.constraint_fns import abs_loss_equality, tpr_equality

# the protected attribute is "Race" (RAC1P)
# we put a pairwise constraint on each combination of subgroups
constraint_bound = 0.01
constraints = []
for gr1, gr2 in combinations(group_indices_train, 2):
    c = FairnessConstraint(
        dataset=dataset_train,
        group_indices=[gr1, gr2],
        # subtract bound from absolute loss difference to bring constraint to form $c \leq 0$
        # also implemented are fairret wrappers, e.g. equal TPR
        fn=lambda model, samples: abs_loss_equality(torch.nn.BCEWithLogitsLoss(), model, samples) - constraint_bound,
        batch_size=c_batch_size,
        device=device
    )

    constraints.append(c)

print(f'Number of constraints: {len(constraints)}')

Number of constraints: 15


In [ ]:
# helper function to analyze model performance

def model_stats(model, features, labels, groups, constraints, constraint_bound):
    with torch.inference_mode():
        gr_ind = list(combinations(groups, 2))
        vals = []
        acc_dif = []
        for i, c in enumerate(constraints):
            idx1, idx2 = gr_ind[i]
            val = c.eval(model, [(features[idx1], labels[idx1]), (features[idx2], labels[idx2])]) + constraint_bound
            vals.append(val.cpu().numpy().item())

            logits1 = model(features[idx1])
            logits2 = model(features[idx2])
            outs1 = torch.nn.functional.sigmoid(logits1).cpu().numpy()
            outs2 = torch.nn.functional.sigmoid(logits2).cpu().numpy()
            preds1 = (outs1.T > 0.5).astype(float)
            preds2 = (outs2.T > 0.5).astype(float) 
            acc1 = np.mean(preds1 == labels[idx1].cpu().numpy())
            acc2 = np.mean(preds2 == labels[idx2].cpu().numpy())
            acc_dif.append(abs(acc1-acc2))

        print(f'constraints (should be <= {constraint_bound}):')
        print(np.round(vals, decimals=3))
        print(f'c mean: {np.mean(vals)}')
        print(f'c min: {np.min(vals)}')
        print(f'c max: {np.max(vals)}')
        print('---')

        logits = model(features)
        outs = torch.nn.functional.sigmoid(logits).cpu().numpy()
        preds = (outs.T > 0.5).astype(float)
        acc = np.sum(preds == labels.cpu().numpy())/len(labels)
        print(f'accuracy: {acc}')
        print('accuracy abs. difference:')
        print(np.round(acc_dif, decimals=3))
        print(f'acc abs dif mean: {np.mean(acc_dif)}')
        print(f'acc abs dif min: {np.min(acc_dif)}')
        print(f'acc abs dif max: {np.max(acc_dif)}')

---
---

For comparison, let us first train a model **without constraints**.

Define a model:

In [ ]:
from torch.nn import Sequential
hsize1 = 64
hsize2 = 32
model_uncon = Sequential(
    torch.nn.Linear(features_train.shape[1], hsize1),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize1, hsize2),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize2, 1)
).to(device)

And start training:

In [ ]:
from torch.optim import Adam

loader = torch.utils.data.DataLoader(dataset_train, batch_size=32, shuffle=(device != 'cuda'))
loss = torch.nn.BCEWithLogitsLoss()
optimizer = Adam(model_uncon.parameters())
epochs = 100

for epoch in range(epochs):
    losses = []
    for batch_feat, batch_label in loader:
        optimizer.zero_grad()

        logit = model_uncon(batch_feat)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logit.squeeze(), batch_label)
        loss.backward()

        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch: {epoch}, loss: {np.mean(losses)}")

Epoch: 0, loss: 0.46349757064932157
Epoch: 1, loss: 0.4295002950488457
Epoch: 2, loss: 0.4227255582809448
Epoch: 3, loss: 0.4194903533040945
Epoch: 4, loss: 0.4161933082421975
Epoch: 5, loss: 0.41442626488528084
Epoch: 6, loss: 0.4117933259611683
Epoch: 7, loss: 0.409918974951974
Epoch: 8, loss: 0.4095359429317926
Epoch: 9, loss: 0.40733223824229625
Epoch: 10, loss: 0.4060150348315282
Epoch: 11, loss: 0.40542341246535735
Epoch: 12, loss: 0.404065324525748
Epoch: 13, loss: 0.40323235995934475
Epoch: 14, loss: 0.4016180704015174
Epoch: 15, loss: 0.4020076899988843
Epoch: 16, loss: 0.40059379511512816
Epoch: 17, loss: 0.39994363406939165
Epoch: 18, loss: 0.39820664409281953
Epoch: 19, loss: 0.398150879656896
Epoch: 20, loss: 0.3973517100592809
Epoch: 21, loss: 0.39707725824389073
Epoch: 22, loss: 0.3955042596041624
Epoch: 23, loss: 0.3960753854563726
Epoch: 24, loss: 0.39501276216469705
Epoch: 25, loss: 0.3946579004571374
Epoch: 26, loss: 0.39336919508475277
Epoch: 27, loss: 0.39273399287

Let's now analyze how well the **unconstrained** model does in terms of constraints:

In [ ]:
print('TRAIN')
model_stats(model_uncon, features_train, labels_train, group_indices_train, constraints, constraint_bound)

TRAIN
constraints (should be <= 0.01):
[0.097 0.015 0.062 0.183 0.055 0.081 0.035 0.086 0.041 0.047 0.168 0.04
 0.121 0.007 0.128]
c mean: 0.0777715116739273
c min: 0.006703734397888184
c max: 0.1832037717103958
---
accuracy: 0.8240424195911533
accuracy abs. difference:
[0.063 0.004 0.022 0.102 0.023 0.059 0.041 0.038 0.041 0.018 0.097 0.018
 0.08  0.001 0.079]
acc abs dif mean: 0.045657815374404734
acc abs dif min: 0.0006777088020819555
acc abs dif max: 0.10155016303823039


In [ ]:
print('TEST')
model_stats(model_uncon, features_test, labels_test, group_indices_test, constraints, constraint_bound)

TEST
constraints (should be <= 0.01):
[0.079 0.057 0.112 0.03  0.075 0.137 0.191 0.109 0.004 0.055 0.027 0.132
 0.082 0.187 0.105]
c mean: 0.09223249951998393
c min: 0.004368424415588379
c max: 0.19137758016586304
---
accuracy: 0.794921875
accuracy abs. difference:
[0.022 0.016 0.011 0.083 0.043 0.038 0.033 0.061 0.021 0.005 0.099 0.059
 0.094 0.054 0.04 ]
acc abs dif mean: 0.04521909230893753
acc abs dif min: 0.004741402801242578
acc abs dif max: 0.09864360424289464


---
---

Now let us train the same model with one of the **constrained** training algorithms:

In [ ]:
from humancompatible.train.algorithms import SSLALM, SSG
from torch.nn import Sequential

hsize1 = 64
hsize2 = 32
model = Sequential(
    torch.nn.Linear(features_train.shape[1], hsize1),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize1, hsize2),
    torch.nn.ReLU(),
    torch.nn.Linear(hsize2, 1)
)

optimizer = SSG(
    net=model,
    data=dataset_train,
    loss=torch.nn.BCEWithLogitsLoss(),
    constraints=constraints
)

history = optimizer.optimize(
    max_runtime=180,
    batch_size=32,
    seed=42,
    device=device,
    ctol=1.0,
    f_stepsize_rule='dimin',
    f_stepsize=0.1,
    c_stepsize_rule='dimin',
    c_stepsize=0.1,
    verbose=False
    )

In [ ]:
print('TRAIN')
model_stats(model, features_train, labels_train, group_indices_train, constraints, constraint_bound)

TRAIN
constraints (should be <= 0.01):
[0.008 0.01  0.002 0.009 0.01  0.002 0.01  0.001 0.002 0.011 0.001 0.001
 0.011 0.012 0.002]
c mean: 0.006037203470865885
c min: 0.0006375312805175781
c max: 0.012136995792388916
---
accuracy: 0.7662038652061676
accuracy abs. difference:
[0.05  0.026 0.071 0.121 0.037 0.024 0.021 0.071 0.012 0.046 0.095 0.012
 0.05  0.034 0.084]
acc abs dif mean: 0.050338703671629736
acc abs dif min: 0.011668145409021724
acc abs dif max: 0.12120685429061662


In [ ]:
print('TEST')
model_stats(model, features_test, labels_test, group_indices_test, constraints, constraint_bound)

TEST
constraints (should be <= 0.01):
[0.005 0.001 0.021 0.007 0.014 0.006 0.026 0.012 0.009 0.02  0.006 0.015
 0.014 0.035 0.021]
c mean: 0.0140508770942688
c min: 0.0007504820823669434
c max: 0.03483313322067261
---
accuracy: 0.7642299107142857
accuracy abs. difference:
[0.056 0.002 0.006 0.06  0.041 0.054 0.05  0.003 0.015 0.004 0.057 0.039
 0.053 0.035 0.018]
acc abs dif mean: 0.033019134885242975
acc abs dif min: 0.0023696482327177915
acc abs dif max: 0.059619158525802796
